# Model fine-tuning
### XLM-RoBERTa and mDeBERTa models fine-tuning on MultiCW and OOD datasets

## Initialization

#### Setup project paths:

In [1]:
import os
import sys
from os.path import join, exists
from py_markdown_table.markdown_table import markdown_table

from tqdm.notebook import tqdm

# Enable tqdm for pandas
tqdm.pandas()

# ANSI Highlighting: https://stackoverflow.com/a/21786287
h_red = '\x1b[1;30;41m'
h_green = '\x1b[1;30;42m'
h_yellow = '\x1b[1;30;43m'
h_stop = '\x1b[0m'

## Setup project paths:
project_path = os.getcwd()
models_path = join(project_path, "Models")

datasets_path = join(project_path, "Source datasets")
multicw_path = join(project_path, 'Final-dataset')
multiclaim_path = join(datasets_path, "MultiClaim")
lesa_dst_dir = join(datasets_path, 'LESA-EACL-2021')
print('done')

done


## Datasets
Loading the MultiCW and OOD datasets for the purpose of models fine-tuning and their evaluation.

In [2]:
# Load MultiCW model
import pandas as pd
from os.path import join
languages = pd.read_csv(join('Final-dataset', 'multicw-full.csv'))['lang'].unique()

multicw_path = join("Final-dataset")
multicw_train = pd.read_csv(join(multicw_path, "multicw-train.csv")).astype({'label':'int'})
multicw_dev = pd.read_csv(join(multicw_path, "multicw-dev.csv")).astype({'label':'int'})
multicw_test = pd.read_csv(join(multicw_path, "multicw-test.csv")).astype({'label':'int'})
multicw_test = pd.read_csv(join(multicw_path, "multicw-test.csv")).astype({'text':'str'})
multicw_ood = pd.read_csv(join(multicw_path, "multicw-ood.csv")).astype({'label':'int'})
multicw_ood['style'] = multicw_ood['style'].replace('structured', 'struc')
multicw_ood['text'] = multicw_ood['text'].fillna("").astype(str)

print(f'Loaded MultiCW:')
print(f'Train set: {multicw_train.shape[0]}')
print(f'Dev set: {multicw_dev.shape[0]}')
print(f'Test set: {multicw_test.shape[0]}')
print(f'Out-of-dist set: {multicw_ood.shape[0]}')

Loaded MultiCW:
Train set: 88001
Dev set: 14823
Test set: 14823
Out-of-dist set: 27761


## Models
Implementation of the used models.

### XLM-RoBERTa

In [3]:
import sys
import os
import shutil
import warnings
from typing import Callable
import numpy as np
import pandas as pd

import keras_hub
import keras

from keras_preprocessing.sequence import pad_sequences
from pandas import DataFrame
from sklearn.metrics import classification_report
from keras.optimizers import Adam
from tqdm import tqdm
from transformers import BertTokenizer, AutoTokenizer


class XLMRobertaModel():
    """ Model finetuning and inference on the LESA dataset using XLMRoberta """

    def __init__(self):
        self.max_len = 256
        self.batch_size = 32

        self.final_model = None

        self.preprocessor = keras_hub.models.TextClassifierPreprocessor.from_preset(
            'xlm_roberta_base_multi',
            sequence_length=self.max_len
        )

    def load_model(self, model_name) -> bool:
        try:
            # 1. Instantiate the model with the known preset (architecture)
            self.final_model = keras_hub.models.XLMRobertaTextClassifier.from_preset(
                'xlm_roberta_base_multi',
                num_classes=2,
                preprocessor=self.preprocessor,
                dropout=0.2
            )
    
            # 2. Load the weights manually from the path
            weights_path = os.path.join('Models', f'{model_name}.weights.h5')
            self.final_model.load_weights(weights_path)
    
            print('Model loaded successfully.')
            return True
        except Exception as e:
            print(f"Error loading model: {e}")
            return False
    
    def detect_claims(self, test_set: DataFrame, verbose=False) -> tuple:
        """Performs inference on the testing data and evaluates results."""

        print('Running classification:')
        metrics = self.final_model.predict(x=test_set['text'].to_numpy(), batch_size=self.batch_size)

        # Testing
        results = np.argmax(metrics, axis=1)
        print('Done.')
        # Print sentences with classifications
        if verbose:
            for i, text in enumerate(test_set['text']):
                print("LESA classification: '{text}' {c} a claim.".
                      format(text=text, c=gh_start + "is" if results[i] == 1 else rh_start + "is not") + h_stop)

        # Compare against ground-truth
        ground_truth = test_set['label'].to_numpy()
        report = classification_report(ground_truth, results, output_dict=True)
        report_str = str(classification_report(ground_truth, results))

        return report, report_str

    def train_model(self, train_set: DataFrame, dev_set: DataFrame, epochs=1, learn_rate=3e-5, model_name='', lang='en'):
        """
        Train the XLMRoberta model with the given parameters.

        :param learn_rate: Learning rate.
        :param train_set: Training dataset.
        :param dev_set: Validation dataset.
        :param epochs: Number of training epochs.
        :param model_name: Model will be saved to the directory named by this value.If left blank, the model won't save.
        :param lang: Training dataset language(s). Needed for naming conventions.
        :return: Trained model.
        """

        model_name_path = f"{model_name}-{lang}-{epochs}e"
        path = os.path.join('Models', 'LESA', 'models', model_name_path)
        if not os.path.exists(path):
            os.makedirs(path)


        self.final_model = keras_hub.models.XLMRobertaTextClassifier.from_preset(
            'xlm_roberta_base_multi',
            num_classes=len(train_set['label'].unique()),
            preprocessor=self.preprocessor,
            dropout=0.2
        )

        for layer in self.final_model.layers:
            layer.trainable = True

        self.final_model.compile(
            loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            optimizer=keras.optimizers.Adam(learn_rate),
            jit_compile=True,
        )

        self.final_model.fit(x=train_set['text'].to_numpy(), y=train_set['label'].to_numpy(), batch_size=self.batch_size,
                             epochs=epochs, validation_data=(dev_set['text'].to_numpy(), dev_set['label'].to_numpy()))

        model_save_path = os.path.join(path, 'model.keras')
        self.final_model.save(model_save_path)

    def inference(self, texts: list[str]) -> list[bool]:
        """Returns a list of booleans: True if classified as a claim (class 1), else False."""
        if not isinstance(texts, list):
            raise ValueError("Input must be a list of strings.")
        if not all(isinstance(t, str) for t in texts):
            raise ValueError("All items in input list must be strings.")
    
        predictions = self.final_model.predict(texts)  # Must be list[str]
        classifications = np.argmax(predictions, axis=1)  # shape (batch_size,)
        
        return (classifications == 1).tolist()


2025-10-01 13:45:57.897642: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-10-01 13:45:57.922397: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-10-01 13:45:57.922454: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-10-01 13:45:57.951995: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-01 13:45:58.731523: W tensorflow/compiler/tf

### mDeBERTa

In [4]:
import sys
import os
import shutil
import warnings
from typing import Callable
import numpy as np
import pandas as pd

import keras_hub
import keras

from keras_preprocessing.sequence import pad_sequences
from pandas import DataFrame
from sklearn.metrics import classification_report
from keras.optimizers import Adam
from tqdm import tqdm
from transformers import BertTokenizer, AutoTokenizer
from tensorflow.keras.optimizers.schedules import CosineDecay


class MDeBertaModel():
    """  """

    def __init__(self, model_name=''):
        self.max_len = 256
        self.batch_size = 32
        self.final_model = None
        
        self.preprocessor = keras_hub.models.TextClassifierPreprocessor.from_preset(
            'deberta_v3_base_multi',
            sequence_length=self.max_len
        )

    def load_model(self, model_name) -> bool:
        try:
            model_path = os.path.join('Models', model_name + '.keras')
    
            if not os.path.exists(model_path):
                raise FileNotFoundError(f"Saved model not found at: {model_path}")
    
            print(f"Loading full model from: {model_path}")
            self.final_model = keras.models.load_model(model_path)
            print("Model loaded successfully.")
            return True
        except Exception as e:
            print(f"Error loading model: {str(e)}")
            return False
            
    def detect_claims(self, test_set: DataFrame, verbose=False) -> tuple:
        """Performs inference on the testing data and evaluates results."""

        print('Running classification:')
        metrics = self.final_model.predict(x=test_set['text'].to_numpy(), batch_size=self.batch_size)

        # Testing
        results = np.argmax(metrics, axis=1)
        print('Done.')
        # Print sentences with classifications
        if verbose:
            for i, text in enumerate(test_set['text']):
                print("LESA classification: '{text}' {c} a claim.".
                      format(text=text, c=gh_start + "is" if results[i] == 1 else rh_start + "is not") + h_stop)

        # Compare against ground-truth
        ground_truth = test_set['label'].to_numpy()
        report = classification_report(ground_truth, results, output_dict=True)
        report_str = str(classification_report(ground_truth, results))

        return report, report_str

    def train_model(self, train_set: DataFrame, dev_set: DataFrame, epochs=1, learn_rate=3e-5, model_name='', lang='en', final_learn_rate_fraction=0.5):
        """
        Train the mDeBerta model with the given parameters.
        :param learn_rate: Learning rate.
        :param train_set: Training dataset.
        :param dev_set: Validation dataset.
        :param epochs: Number of training epochs.
        :param model_name: Model will be saved to the directory named by this value.If left blank, the model won't save.
        :param lang: Training dataset language(s). Needed for naming conventions.
        :return: Trained model.
        """

        self.final_model = keras_hub.models.DebertaV3Classifier.from_preset(
            'deberta_v3_base_multi',
            num_classes=len(set(train_set['label'].unique())),
            preprocessor=self.preprocessor,
            dropout=0.2
        )

        for layer in self.final_model.layers:
            layer.trainable = True

        initial_learning_rate = learn_rate
        decay_steps = int(len(train_set) // self.batch_size * epochs)   # Number of steps over which the decay is applied
        alpha = final_learn_rate_fraction  # Minimum learning rate as a fraction of initial_learning_rate

        lr_schedule = CosineDecay(
            initial_learning_rate=initial_learning_rate,
            decay_steps=decay_steps,
            alpha=alpha
        )

        self.final_model.compile(
            loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
            jit_compile=True
        )

        self.final_model.fit(x=train_set['text'].to_numpy(), y=train_set['label'].to_numpy(), batch_size=self.batch_size,
                             epochs=epochs, validation_data=(dev_set['text'].to_numpy(), dev_set['label'].to_numpy()))

        model_save_path = os.path.join(path, f"{model_name}-{lang}-{epochs}e.keras")
        self.final_model.save(model_save_path)
        
    def inference(self, texts: list[str]) -> list[bool]:
        """Returns a list of booleans: True if classified as a claim (class 1), else False."""
        if not isinstance(texts, list):
            raise ValueError("Input must be a list of strings.")
        if not all(isinstance(t, str) for t in texts):
            raise ValueError("All items in input list must be strings.")
    
        predictions = self.final_model.predict(texts)  # Must be list[str]
        classifications = np.argmax(predictions, axis=1)  # shape (batch_size,)
        
        return (classifications == 1).tolist()

## Experiments

### Models fine-tuning on MultiCW dataset
- Fine-tuning of xlm-RoBERTa, mDeBERTa anb LESA models on MultiCW train set
- Evaluation on MultiCW test set

#### XLM-RoBERTa model:
<table>
<tr>
<td>

| Class            | Precision | Recall | F1-score | Support   |
| ---------------- | --------- | ------ | -------- | --------- |
| 0                | 0.87      | 0.88   | 0.87     | 9269      |
| 1                | 0.88      | 0.87   | 0.87     | 9175      |
| **Accuracy**     |           |        | **0.87** | **18444** |
| **Macro avg**    | 0.87      | 0.87   | 0.87     | 18444     |
| **Weighted avg** | 0.87      | 0.87   | 0.87     | 18444     |
                    MultiCW Overall.
</td>
<td>

| Class            | Precision | Recall | F1-score | Support  |
| ---------------- | --------- | ------ | -------- | -------- |
| 0                | 0.81      | 0.82   | 0.81     | 4744     |
| 1                | 0.81      | 0.80   | 0.81     | 4650     |
| **Accuracy**     |           |        | **0.81** | **9394** |
| **Macro avg**    | 0.81      | 0.81   | 0.81     | 9394     |
| **Weighted avg** | 0.81      | 0.81   | 0.81     | 9394     |
                MultiCW Noisy Part.
</td>
<td>

| Class            | Precision | Recall | F1-score | Support  |
| ---------------- | --------- | ------ | -------- | -------- |
| 0                | 0.93      | 0.94   | 0.94     | 4525     |
| 1                | 0.94      | 0.93   | 0.93     | 4525     |
| **Accuracy**     |           |        | **0.93** | **9050** |
| **Macro avg**    | 0.93      | 0.93   | 0.93     | 9050     |
| **Weighted avg** | 0.93      | 0.93   | 0.93     | 9050     |
                MultiCW Structured Part.
</td>
</tr>
</table>

#### mDeBERTa model:
<table>
<tr>
<td>

| Class            | Precision | Recall | F1-score | Support   |
| ---------------- | --------- | ------ | -------- | --------- |
| 0                | 0.89      | 0.86   | 0.87     | 9269      |
| 1                | 0.86      | 0.89   | 0.88     | 9175      |
| **Accuracy**     |           |        | **0.88** | **18444** |
| **Macro avg**    | 0.88      | 0.88   | 0.88     | 18444     |
| **Weighted avg** | 0.88      | 0.88   | 0.88     | 18444     |
                    MultiCW Overall.
</td>
<td>

| Class            | Precision | Recall | F1-score | Support  |
| ---------------- | --------- | ------ | -------- | -------- |
| 0                | 0.84      | 0.80   | 0.82     | 4744     |
| 1                | 0.81      | 0.84   | 0.82     | 4650     |
| **Accuracy**     |           |        | **0.82** | **9394** |
| **Macro avg**    | 0.82      | 0.82   | 0.82     | 9394     |
| **Weighted avg** | 0.82      | 0.82   | 0.82     | 9394     |
                MultiCW Noisy Part.
</td>
<td>

| Class            | Precision | Recall | F1-score | Support  |
| ---------------- | --------- | ------ | -------- | -------- |
| 0                | 0.95      | 0.91   | 0.93     | 4525     |
| 1                | 0.92      | 0.95   | 0.93     | 4525     |
| **Accuracy**     |           |        | **0.93** | **9050** |
| **Macro avg**    | 0.93      | 0.93   | 0.93     | 9050     |
| **Weighted avg** | 0.93      | 0.93   | 0.93     | 9050     |
                MultiCW Structured Part.
</td>
</tr>
</table>

In [6]:
model = None
detector = None
models = ['xlm', 'mdb']

for model in models:
    if model == 'xlm':
        print(f'{h_green}XLM-RoBERTa model:{h_stop}')
        detector = XLMRobertaModel()
    if model == 'mdb':
        print(f'{h_green}mDBERTa model:{h_stop}')
        detector = MDeBertaModel()

    if not detector.load_model(model_name=f'{model}-multicw-2e6-5e'):
        print(f'{h_yellow}No model found. Initiating fine-tuning:{h_stop}')
        # Note: Works well with a small learning rate (e.g. 3e-6)
        detector.train_model(multicw_train, multicw_dev, epochs=5, learn_rate=2e-6, lang='en', model_name=f'{model}-multicw')
    
    print(f'{h_yellow}MultiCW overall:{h_stop}')
    _, report = detector.detect_claims(multicw_test)
    print(report)
    
    test_noisy = multicw_test.loc[multicw_test['style']=='noisy']
    _, report = detector.detect_claims(test_noisy, verbose=False)
    print(f'{h_yellow}MultiCW Noisy Part:{h_stop}')
    print(report)
    
    test_strut = multicw_test.loc[multicw_test['style']=='struct']
    _, report = detector.detect_claims(test_strut, verbose=False)
    print(f'{h_yellow}MultiCW Structured Part:{h_stop}')
    print(report)

XLM-RoBERTa model:


2025-09-01 07:29:50.732261: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-09-01 07:29:50.787211: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-09-01 07:29:50.791843: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Model loaded successfully.
MultiCW overall:
Running classification:


I0000 00:00:1756704598.034464 1441906 service.cc:145] XLA service 0x7fc4b002cdc0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1756704598.034524 1441906 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2025-09-01 07:29:58.117842: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-09-01 07:29:58.332846: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907
I0000 00:00:1756704599.611815 1441982 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_2670', 348 bytes spill stores, 356 bytes spill loads



  1/577 ━━━━━━━━━━━━━━━━━━━━ 34:16 4s/step

...

294/294 ━━━━━━━━━━━━━━━━━━━━ 61s 206ms/step
Done.
MultiCW Noisy Part:
              precision    recall  f1-score   support

           0       0.81      0.82      0.81      4744
           1       0.81      0.80      0.81      4650

    accuracy                           0.81      9394
   macro avg       0.81      0.81      0.81      9394
weighted avg       0.81      0.81      0.81      9394

Running classification:
282/283 ━━━━━━━━━━━━━━━━━━━━ 0s 198ms/step

I0000 00:00:1756704835.888722 1442280 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_2688', 60 bytes spill stores, 60 bytes spill loads

I0000 00:00:1756704836.010144 1442286 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_2688', 352 bytes spill stores, 356 bytes spill loads



283/283 ━━━━━━━━━━━━━━━━━━━━ 59s 206ms/step
Done.
MultiCW Structured Part:
              precision    recall  f1-score   support

           0       0.93      0.94      0.94      4525
           1       0.94      0.93      0.93      4525

    accuracy                           0.93      9050
   macro avg       0.93      0.93      0.93      9050
weighted avg       0.93      0.93      0.93      9050

mDBERTa model:
Loading full model from: Models/mdb-multicw-2e6-5e.keras


/home/hyben/.conda/envs/MultiCW-finetune/lib/python3.10/site-packages/keras/src/saving/serialization_lib.py:734: UserWarning: `compile()` was not called as part of model loading because the model's `compile()` method is custom. All subclassed Models that have `compile()` overridden should also override `get_compile_config()` and `compile_from_config(config)`. Alternatively, you can call `compile()` manually after loading.
  instance.compile_from_config(compile_config)
2025-09-01 07:34:13.255647: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 771072000 exceeds 10% of free system memory.
/home/hyben/.conda/envs/MultiCW-finetune/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 406 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model loaded successfully.
MultiCW overall:
Running classification:
577/577 ━━━━━━━━━━━━━━━━━━━━ 201s 339ms/step
Done.
              precision    recall  f1-score   support

           0       0.89      0.86      0.87      9269
           1       0.86      0.89      0.88      9175

    accuracy                           0.88     18444
   macro avg       0.88      0.88      0.88     18444
weighted avg       0.88      0.88      0.88     18444

Running classification:
294/294 ━━━━━━━━━━━━━━━━━━━━ 100s 341ms/step
Done.
MultiCW Noisy Part:
              precision    recall  f1-score   support

           0       0.84      0.80      0.82      4744
           1       0.81      0.84      0.82      4650

    accuracy                           0.82      9394
   macro avg       0.82      0.82      0.82      9394
weighted avg       0.82      0.82      0.82      9394

Running classification:
283/283 ━━━━━━━━━━━━━━━━━━━━ 97s 343ms/step
Done.
MultiCW Structured Part:
              precision    recall

### Evaluation of models on each language of the MultiCW dataset

In [5]:
model = None
detector = None
models = ['xlm', 'mdb']

for model in models:
    if model == 'xlm':
        print(f'{h_green}XLM-RoBERTa model:{h_stop}')
        detector = XLMRobertaModel()
    if model == 'mdb':
        print(f'{h_green}mDBERTa model:{h_stop}')
        detector = MDeBertaModel()

    if not detector.load_model(model_name=f'{model}-multicw-2e6-5e'):
        print(f'{h_yellow}No model found. Initiating fine-tuning:{h_stop}')
        # Note: Works well with a small learning rate (e.g. 3e-6)
        detector.train_model(multicw_train, multicw_dev, epochs=5, learn_rate=2e-6, lang='en', model_name=f'{model}-multicw')

    languages = multicw_test['lang'].unique()

    for lang in languages:
        print(f'{h_yellow}Language: {lang}:{h_stop}')
        _, report = detector.detect_claims(multicw_test[multicw_test['lang'] == lang])
        print(report)

XLM-RoBERTa model:


2025-09-25 13:48:52.439519: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-09-25 13:48:52.484092: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-09-25 13:48:52.488673: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Model loaded successfully.
Language: sk:
Running classification:


I0000 00:00:1758800941.258255   15687 service.cc:145] XLA service 0x7fbda802de10 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1758800941.258297   15687 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2025-09-25 13:49:01.364506: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-09-25 13:49:01.617889: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907
I0000 00:00:1758800943.218445   15765 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_2670', 348 bytes spill stores, 356 bytes spill loads



 1/38 ━━━━━━━━━━━━━━━━━━━━ 2:39 4s/step

...

34/34 ━━━━━━━━━━━━━━━━━━━━ 16s 477ms/step
Done.
              precision    recall  f1-score   support

         0.0       0.71      0.93      0.81       530
         1.0       0.90      0.62      0.73       530

    accuracy                           0.77      1060
   macro avg       0.80      0.77      0.77      1060
weighted avg       0.80      0.77      0.77      1060

Language: ru:
Running classification:
38/38 ━━━━━━━━━━━━━━━━━━━━ 13s 334ms/step
Done.
              precision    recall  f1-score   support

         0.0       0.91      0.96      0.93       597
         1.0       0.95      0.90      0.93       597

    accuracy                           0.93      1194
   macro avg       0.93      0.93      0.93      1194
weighted avg       0.93      0.93      0.93      1194

Language: uk:
Running classification:
37/38 ━━━━━━━━━━━━━━━━━━━━ 0s 341ms/step

I0000 00:00:1758801230.747434   17313 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_135', 12 bytes spill stores, 12 bytes spill loads

I0000 00:00:1758801231.445656   17318 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_133', 12 bytes spill stores, 12 bytes spill loads

I0000 00:00:1758801231.643947   17313 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_135', 96 bytes spill stores, 96 bytes spill loads

I0000 00:00:1758801232.098224   17328 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_135', 268 bytes spill stores, 268 bytes spill loads

I0000 00:00:1758801232.357707   17317 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_133', 96 bytes spill stores, 96 bytes spill loads

I0000 00:00:1758801232.671572   1

38/38 ━━━━━━━━━━━━━━━━━━━━ 18s 468ms/step
Done.
              precision    recall  f1-score   support

         0.0       0.86      0.95      0.90       595
         1.0       0.94      0.85      0.89       595

    accuracy                           0.90      1190
   macro avg       0.90      0.90      0.90      1190
weighted avg       0.90      0.90      0.90      1190

Language: zh:
Running classification:
33/33 ━━━━━━━━━━━━━━━━━━━━ 11s 340ms/step
Done.
              precision    recall  f1-score   support

         0.0       0.87      0.91      0.89       517
         1.0       0.90      0.87      0.89       517

    accuracy                           0.89      1034
   macro avg       0.89      0.89      0.89      1034
weighted avg       0.89      0.89      0.89      1034

Language: hi:
Running classification:
38/38 ━━━━━━━━━━━━━━━━━━━━ 13s 344ms/step
Done.
              precision    recall  f1-score   support

         0.0       1.00      0.84      0.91       597
         1.0     

I0000 00:00:1758801345.566506   17766 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_168', 4 bytes spill stores, 4 bytes spill loads

I0000 00:00:1758801345.900553   17775 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_168', 96 bytes spill stores, 96 bytes spill loads

I0000 00:00:1758801346.042456   17770 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_168', 264 bytes spill stores, 264 bytes spill loads

I0000 00:00:1758801346.485220   17771 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_169', 4 bytes spill stores, 4 bytes spill loads

I0000 00:00:1758801346.621448   17773 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_168', 1304 bytes spill stores, 1328 bytes spill loads

I0000 00:00:1758801347.567662   1

30/30 ━━━━━━━━━━━━━━━━━━━━ 16s 537ms/step
Done.
              precision    recall  f1-score   support

         0.0       1.00      0.84      0.91       465
         1.0       0.86      1.00      0.92       465

    accuracy                           0.92       930
   macro avg       0.93      0.92      0.92       930
weighted avg       0.93      0.92      0.92       930

Language: pt:
Running classification:
38/38 ━━━━━━━━━━━━━━━━━━━━ 13s 348ms/step
Done.
              precision    recall  f1-score   support

         0.0       0.99      0.96      0.98       597
         1.0       0.96      0.99      0.98       597

    accuracy                           0.98      1194
   macro avg       0.98      0.98      0.98      1194
weighted avg       0.98      0.98      0.98      1194

Language: de:
Running classification:
38/38 ━━━━━━━━━━━━━━━━━━━━ 14s 357ms/step
Done.
              precision    recall  f1-score   support

         0.0       1.00      0.94      0.97       596
         1.0     

### Out-domain evaluation
- Evaluation of the fine-tuned models on 4 more languages obtained from source datasets.

#### XLM-RoBERTa model:
<table>
<tr>
<td>

| Class            | Precision | Recall | F1-score | Support   |
| ---------------- | --------- | ------ | -------- | --------- |
| 0                | 0.94      | 0.75   | 0.84     | 16000     |
| 1                | 0.76      | 0.95   | 0.85     | 13647     |
| **Accuracy**     |           |        | **0.84** | **29647** |
| **Macro avg**    | 0.85      | 0.85   | 0.84     | 29647     |
| **Weighted avg** | 0.86      | 0.84   | 0.84     | 29647     |
                    MultiCW Overall.
</td>
<td>

| Class            | Precision | Recall | F1-score | Support  |
| ---------------- | --------- | ------ | -------- | -------- |
| 0                | 0.91      | 0.70   | 0.79     | 8000     |
| 1                | 0.76      | 0.93   | 0.84     | 8000     |
| **Accuracy**     |           |        | **0.82** | **16000** |
| **Macro avg**    | 0.83      | 0.82   | 0.81     | 16000     |
| **Weighted avg** | 0.83      | 0.82   | 0.81     | 16000     |
                MultiCW Noisy Part.
</td>
<td>

| Class            | Precision | Recall | F1-score | Support  |
| ---------------- | --------- | ------ | -------- | -------- |
| 0                | 0.97      | 0.80   | 0.88     | 8000     |
| 1                | 0.78      | 0.97   | 0.86     | 5647     |
| **Accuracy**     |           |        | **0.87** | **13647** |
| **Macro avg**    | 0.87      | 0.88   | 0.87     | 13647     |
| **Weighted avg** | 0.89      | 0.87   | 0.87     | 13647     |
                MultiCW Structured Part.
</td>
</tr>
</table>

#### mDBERTa model:
<table>
<tr>
<td>

| Class            | Precision | Recall | F1-score | Support   |
| ---------------- | --------- | ------ | -------- | --------- |
| 0                | 0.97      | 0.74   | 0.84     | 16000     |
| 1                | 0.76      | 0.97   | 0.85     | 13647     |
| **Accuracy**     |           |        | **0.85** | **29647** |
| **Macro avg**    | 0.86      | 0.86   | 0.85     | 29647     |
| **Weighted avg** | 0.87      | 0.85   | 0.85     | 29647     |
                    MultiCW Overall.
</td>
<td>

| Class            | Precision | Recall | F1-score | Support  |
| ---------------- | --------- | ------ | -------- | -------- |
| 0                | 0.95      | 0.68   | 0.80     | 8000     |
| 1                | 0.75      | 0.96   | 0.85     | 8000     |
| **Accuracy**     |           |        | **0.82** | **16000** |
| **Macro avg**    | 0.85      | 0.82   | 0.82     | 16000     |
| **Weighted avg** | 0.85      | 0.82   | 0.82     | 16000     |
                MultiCW Noisy Part.
</td>
<td>

| Class            | Precision | Recall | F1-score | Support  |
| ---------------- | --------- | ------ | -------- | -------- |
| 0                | 0.98      | 0.79   | 0.88     | 8000     |
| 1                | 0.77      | 0.98   | 0.86     | 5647     |
| **Accuracy**     |           |        | **0.87** | **13647** |
| **Macro avg**    | 0.88      | 0.89   | 0.87     | 13647     |
| **Weighted avg** | 0.90      | 0.87   | 0.87     | 13647     |
                MultiCW Structured Part.
</td>
</tr>
</table>


In [5]:
model = None
detector = None
models = ['xlm', 'mdb']

for model in models:
    if model == 'xlm':
        print(f'{h_green}XLM-RoBERTa model:{h_stop}')
        detector = XLMRobertaModel()
    if model == 'mdb':
        print(f'{h_green}mDBERTa model:{h_stop}')
        detector = MDeBertaModel()

    if not detector.load_model(model_name=f'{model}-multicw-2e6-5e'):
        print(f'{h_yellow}No model found. Initiating fine-tuning:{h_stop}')
        # Note: Works well with a small learning rate (e.g. 3e-6)
        detector.train_model(multicw_train, multicw_dev, epochs=5, learn_rate=2e-6, lang='en', model_name=f'{model}-multicw')
    
    print(f'{h_yellow}MultiCW overall:{h_stop}')
    _, report = detector.detect_claims(multicw_ood)
    print(report)
    
    test_noisy = multicw_ood.loc[multicw_ood['style']=='noisy']
    print(f'{h_yellow}MultiCW Noisy Part:{h_stop}')
    _, report = detector.detect_claims(test_noisy, verbose=False)
    print(report)
    
    test_strut = multicw_ood.loc[multicw_ood['style']=='struc']
    print(f'{h_yellow}MultiCW Structured Part:{h_stop}')
    _, report = detector.detect_claims(test_strut, verbose=False)
    print(report)

XLM-RoBERTa model:


2025-10-01 13:46:26.855076: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-10-01 13:46:26.980819: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-10-01 13:46:26.983829: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Model loaded successfully.
MultiCW overall:
Running classification:


I0000 00:00:1759319194.657170   23559 service.cc:145] XLA service 0x7f7b980022e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1759319194.657218   23559 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2025-10-01 13:46:34.729265: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-10-01 13:46:34.936323: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907
I0000 00:00:1759319196.303246   23640 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_2670', 348 bytes spill stores, 356 bytes spill loads



  1/868 ━━━━━━━━━━━━━━━━━━━━ 53:01 4s/step

...

427/427 ━━━━━━━━━━━━━━━━━━━━ 88s 205ms/step
Done.
              precision    recall  f1-score   support

           0       0.97      0.80      0.88      8000
           1       0.78      0.97      0.86      5647

    accuracy                           0.87     13647
   macro avg       0.87      0.88      0.87     13647
weighted avg       0.89      0.87      0.87     13647

mDBERTa model:
Loading full model from: Models/mdb-multicw-2e6-5e.keras


/home/hyben/.conda/envs/MultiCW-finetune/lib/python3.10/site-packages/keras/src/saving/serialization_lib.py:734: UserWarning: `compile()` was not called as part of model loading because the model's `compile()` method is custom. All subclassed Models that have `compile()` overridden should also override `get_compile_config()` and `compile_from_config(config)`. Alternatively, you can call `compile()` manually after loading.
  instance.compile_from_config(compile_config)
2025-10-01 13:52:49.381804: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 771072000 exceeds 10% of free system memory.
/home/hyben/.conda/envs/MultiCW-finetune/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 406 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model loaded successfully.
MultiCW overall:
Running classification:
868/868 ━━━━━━━━━━━━━━━━━━━━ 304s 345ms/step
Done.
              precision    recall  f1-score   support

           0       0.97      0.79      0.87     15997
           1       0.77      0.97      0.86     11764

    accuracy                           0.86     27761
   macro avg       0.87      0.88      0.86     27761
weighted avg       0.89      0.86      0.86     27761

MultiCW Noisy Part:
Running classification:
441/442 ━━━━━━━━━━━━━━━━━━━━ 0s 350ms/step

I0000 00:00:1759320032.972179   25679 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_169', 1304 bytes spill stores, 1328 bytes spill loads

I0000 00:00:1759320033.061594   25693 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_169', 96 bytes spill stores, 96 bytes spill loads

I0000 00:00:1759320033.109274   25682 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_169', 4 bytes spill stores, 4 bytes spill loads

I0000 00:00:1759320033.264362   25688 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_168', 96 bytes spill stores, 96 bytes spill loads

I0000 00:00:1759320033.363912   25681 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_169', 264 bytes spill stores, 264 bytes spill loads

I0000 00:00:1759320033.608206  

442/442 ━━━━━━━━━━━━━━━━━━━━ 159s 360ms/step
Done.
              precision    recall  f1-score   support

           0       0.96      0.78      0.86      7997
           1       0.77      0.96      0.85      6117

    accuracy                           0.86     14114
   macro avg       0.86      0.87      0.86     14114
weighted avg       0.88      0.86      0.86     14114

MultiCW Structured Part:
Running classification:
427/427 ━━━━━━━━━━━━━━━━━━━━ 151s 354ms/step
Done.
              precision    recall  f1-score   support

           0       0.98      0.79      0.88      8000
           1       0.77      0.98      0.86      5647

    accuracy                           0.87     13647
   macro avg       0.88      0.89      0.87     13647
weighted avg       0.90      0.87      0.87     13647



### Evaluation of models on each language of the Out-of-distribution dataset

In [5]:
model = None
detector = None
models = ['xlm', 'mdb']

for model in models:
    if model == 'xlm':
        print(f'{h_green}XLM-RoBERTa model:{h_stop}')
        detector = XLMRobertaModel()
    if model == 'mdb':
        print(f'{h_green}mDBERTa model:{h_stop}')
        detector = MDeBertaModel()

    if not detector.load_model(model_name=f'{model}-multicw-2e6-5e'):
        print(f'{h_yellow}No model found. Initiating fine-tuning:{h_stop}')
        # Note: Works well with a small learning rate (e.g. 3e-6)
        detector.train_model(multicw_train, multicw_dev, epochs=5, learn_rate=2e-6, lang='en', model_name=f'{model}-multicw')

    languages = multicw_ood['lang'].unique()

    for lang in languages:
        print(f'{h_yellow}Language: {lang}:{h_stop}')
        _, report = detector.detect_claims(multicw_ood[multicw_ood['lang'] == lang])
        print(report)

XLM-RoBERTa model:


2025-09-25 19:50:35.876046: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-09-25 19:50:35.937876: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-09-25 19:50:35.940863: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Model loaded successfully.
Language: nl:
Running classification:


I0000 00:00:1758822643.508835   28888 service.cc:145] XLA service 0x7f791402d3e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1758822643.508880   28888 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2025-09-25 19:50:43.584956: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-09-25 19:50:43.783665: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907
I0000 00:00:1758822645.081311   28970 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_2670', 348 bytes spill stores, 356 bytes spill loads



  1/226 ━━━━━━━━━━━━━━━━━━━━ 13:30 4s/step

...

223/223 ━━━━━━━━━━━━━━━━━━━━ 47s 208ms/step
Done.
              precision    recall  f1-score   support

           0       0.94      0.86      0.90      4000
           1       0.84      0.93      0.88      3123

    accuracy                           0.89      7123
   macro avg       0.89      0.89      0.89      7123
weighted avg       0.89      0.89      0.89      7123

mDBERTa model:
Loading full model from: Models/mdb-multicw-2e6-5e.keras


/home/hyben/.conda/envs/MultiCW-finetune/lib/python3.10/site-packages/keras/src/saving/serialization_lib.py:734: UserWarning: `compile()` was not called as part of model loading because the model's `compile()` method is custom. All subclassed Models that have `compile()` overridden should also override `get_compile_config()` and `compile_from_config(config)`. Alternatively, you can call `compile()` manually after loading.
  instance.compile_from_config(compile_config)
2025-09-25 19:54:16.975197: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 771072000 exceeds 10% of free system memory.
/home/hyben/.conda/envs/MultiCW-finetune/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 406 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model loaded successfully.
Language: nl:
Running classification:
226/226 ━━━━━━━━━━━━━━━━━━━━ 82s 347ms/step
Done.
              precision    recall  f1-score   support

           0       0.93      0.67      0.78      4000
           1       0.69      0.94      0.80      3227

    accuracy                           0.79      7227
   macro avg       0.81      0.80      0.79      7227
weighted avg       0.82      0.79      0.79      7227

Language: my:
Running classification:
228/229 ━━━━━━━━━━━━━━━━━━━━ 0s 332ms/step

I0000 00:00:1758823019.472424   29538 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_183', 4 bytes spill stores, 4 bytes spill loads

I0000 00:00:1758823019.755811   29530 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_183', 264 bytes spill stores, 264 bytes spill loads

I0000 00:00:1758823019.917077   29539 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_182', 4 bytes spill stores, 4 bytes spill loads

I0000 00:00:1758823020.144542   29524 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_183', 96 bytes spill stores, 96 bytes spill loads

I0000 00:00:1758823020.414441   29536 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_183', 1304 bytes spill stores, 1328 bytes spill loads

I0000 00:00:1758823020.750920   2

229/229 ━━━━━━━━━━━━━━━━━━━━ 81s 353ms/step
Done.
              precision    recall  f1-score   support

           0       0.99      0.55      0.71      4000
           1       0.65      1.00      0.78      3297

    accuracy                           0.75      7297
   macro avg       0.82      0.77      0.75      7297
weighted avg       0.84      0.75      0.74      7297

Language: it:
Running classification:
250/250 ━━━━━━━━━━━━━━━━━━━━ 84s 334ms/step
Done.
              precision    recall  f1-score   support

           0       0.98      0.90      0.94      4000
           1       0.90      0.98      0.94      4000

    accuracy                           0.94      8000
   macro avg       0.94      0.94      0.94      8000
weighted avg       0.94      0.94      0.94      8000

Language: mk:
Running classification:
223/223 ━━━━━━━━━━━━━━━━━━━━ 77s 343ms/step
Done.
              precision    recall  f1-score   support

           0       0.97      0.84      0.90      4000
           